# API Keys

In [1]:
from config import METISAI_API_KEY

# Text Collector

In [35]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader, UnstructuredMarkdownLoader

## Collect text files

In [33]:
from pathlib import Path

index_path = "../index/varzesh3.com/news/"

def collect_text_files(root_dir, pattern= "*.md"):
    return list(Path(root_dir).rglob(pattern))

texts_paths = collect_text_files(index_path, "2167*.md")

In [34]:
len(texts_paths)

595

### TEST

In [ ]:
# text_file_path = '../index/varzesh3.com/news/2167552-رکورد-استثنایی-هالند-رونالدو-در-خطر-است.md'

# /Users/cafebazaar/ICT/rag-with-crawler/index/varzesh3.com/football/team/1-ذوب-آهن.md

In [14]:
texts_paths

[PosixPath('../index/varzesh3.com/news/2167276-برد-نفس-گیر-خاتون-در-دربی-بزرگ-لیگ-زنان.md'),
 PosixPath('../index/varzesh3.com/news/2167196-مهم-ترین-فیفادی-جام-جهانی-در-انتظار-سردار-آزمون.md'),
 PosixPath('../index/varzesh3.com/news/2167143-اعتراف-صریح-رونالدو-از-طاس-شدن-وحشت-دارم.md'),
 PosixPath('../index/varzesh3.com/news/2167027-شوخی-زشت-و-عجیب-با-لیگ-برتر-والیبال-ایران.md'),
 PosixPath('../index/varzesh3.com/news/2167259-مورینیو-باعث-فحاشی-به-من-و-خانواده-ام-شد.md'),
 PosixPath('../index/varzesh3.com/news/2167516-تیم-ملی-خیلی-زود-ریکاوری-کرد.md'),
 PosixPath('../index/varzesh3.com/news/2167364-اشلوت-فقط-با-شانس-لیورپول-را-قهرمان-کرد.md'),
 PosixPath('../index/varzesh3.com/news/2167433-درخواست-وایرال-فوتبالیست-فلسطینی-اجابت-شد.md'),
 PosixPath('../index/varzesh3.com/news/2167329-معجزه-فوتبال-در-مقدماتی-جام-جهانی.md'),
 PosixPath('../index/varzesh3.com/news/2167347-ادامه-اختلاف-در-گچساران-تعداد-کادر-زیاد-است.md'),
 PosixPath('../index/varzesh3.com/news/2167348-پرش-بلند-نعمتی-در-سد-

In [97]:
# path = "../index/varzesh3.com/football/team/1-ذوب-آهن.md"
# path = '../index/varzesh3.com/news/2167552-رکورد-استثنایی-هالند-رونالدو-در-خطر-است.md'
path = all_texts_paths[0]

# Open the file in read mode
with open(path, "r") as file:
    content = file.read()  # Read the whole file into a single string

print(content)

## گل گهر 3 - خاتون 4
# برد نفس‌گیر خاتون در دربی بزرگ لیگ زنان
دربی کرمانی‌های فوتبال زنان به سود خاتون بم رقم خورد. 
به گزارش "ورزش سه"، در مهمترین بازی هفته پنجم لیگ برتر فوتبال زنان، تیم گل گهر سیرجان در حالی میزبان 
بازی دو نیمه متفاوت داشت. در نیمه اول، خاتون توانست با ارائه یک بازی سراسر هجومی و برتر به برد پرگل 4 بر یک دست پیدا کند. با این حال، شاگردان مریم جهان نجاتی در نیمه دوم به دنبال تغییر شرایط بودند. آبی‌ها مالکیت بهتری بر بازی داشتند و حتی توانستند با زدن دو گل اختلاف را به حداقل برسانند.
دربی استان کرمان که با شرایطی جذاب دنبال شد، در نهایت خاتون بم را به یک برد نفس‌گیر رساند. نتیجه‌ای که اولین باخت گل‌گهر را رقم زد و صدر جدول را از این تیم گرفت. در عین حال، خاتون را روی دور برد نگه داشت.
حالا خاتون بم و گل گهر سیرجان با امتیاز برابر در 12، به ترتیب اول و دوم جدول هستند و به فکر پیشی گرفتن از یکدیگر در ادامه فصل خواهند بود.
## دیدگاه‌ها
لطفا منتظر بمانید...
## آخرین اخبار
برد نفس‌گیر خاتون در دربی بزرگ لیگ زنان | ورزش سه



In [21]:
c_count = []
for x in texts_paths:
    with open(x, "r") as file:
        content = file.read()
        c_count.append((len(content), x))


In [22]:
c_count.sort()

In [23]:
c_count

[(241,
  PosixPath('../index/varzesh3.com/news/2167112-کاناوارو-مهم-بود-که-با-پیروزی-شروع-کنیم.md')),
 (259,
  PosixPath('../index/varzesh3.com/news/2167015-واکنش-سپاهان-به-شایعه-مذاکره-با-اوسمار.md')),
 (259,
  PosixPath('../index/varzesh3.com/news/2167289-قلعه-نویی-ترکیب-جام-جهانی-را-پیدا-می-کنیم.md')),
 (263,
  PosixPath('../index/varzesh3.com/news/2167079-مربی-روس-ایران-حریف-قدرمندی-برای-ماست.md')),
 (279,
  PosixPath('../index/varzesh3.com/news/2167266-اولین-جایزه-آلگری-در-بازگشت-به-میلان.md')),
 (281,
  PosixPath('../index/varzesh3.com/news/2167202-ستاره-سابق-زنیت-ایران-بسیار-قوی-است.md')),
 (290,
  PosixPath('../index/varzesh3.com/news/2167412-نائوموف-ایران-تحت-پرس-شدید-دچار-استرس-شد.md')),
 (296,
  PosixPath('../index/varzesh3.com/news/2167064-مومنی-آقای-تاجرنیا-برای-استقلال-ورزشگاه-بساز.md')),
 (316,
  PosixPath('../index/varzesh3.com/news/2167573-آرزوی-عجیب-کی-روش-باخت-پنج-بر-صفر-قطر.md')),
 (317,
  PosixPath('../index/varzesh3.com/news/2167074-با-مودریچ-میلان-یک-مالدینی-

In [32]:
loader = TextLoader("../index/varzesh3.com/news/2167482-تکلیف-تیم-ملی-با-خودش-باید-روشن-بشود.md")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
text_splitter.split_text()
docs = text_splitter.split_documents(documents)

TypeError: CharacterTextSplitter.split_text() missing 1 required positional argument: 'text'

In [30]:
len(docs[0].page_content)

8129

In [98]:
loader = TextLoader(path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)

In [16]:
docs

[Document(metadata={'source': '../index/varzesh3.com/news/2167552-رکورد-استثنایی-هالند-رونالدو-در-خطر-است.md'}, page_content='## ارلینگ سریعتر از همه به رکورد 50 گل ملی رسید\n# رکورد استثنایی هالند: رونالدو در خطر است!\nارلینگ هالند با هت\u200cتریک در بازی امشب نروژ در ۴۶ بازی از رکورد ۵۰ گل ملی عبور کرد. رکوردی که حتی کریستیانو رونالدو را هم نگران خواهد کرد.\nبه گزارش ورزش سه، \nبرای مقایسه باید اشاره شود که پیش از او هری کین از این نظر رکورددار بود ولی شاهزاده انگلیسی برای رسیدن به ۵۰ گل به ۷۱ بازی نیاز داشت. و کریستیانو رونالدو؟ او گل پنجاهم ملی\u200cاش را در بازی شماره ۱۱۴ و در جام جهانی ۲۰۱۴ مقابل غنا به ثمر رساند.\nالبته فاصله هالند تا رکورد گلزنی ملی رونالدو هنوز زیاد است؛ کاپیتان پرتغال ۱۴۱ گل ملی دارد و فاصله\u200cای بزرگ میان او و هر بازیکن دیگری دیده می\u200cشود که بعید است به این زودی\u200cها این فاصله پر شود. اما نشانه\u200cها روشن\u200cاند: اگر هالند با همین ریتم ادامه دهد، شاید روزی حتی رکورد مردی را که سال\u200cها دست\u200cنیافتنی به نظر می\u200cرسید، به خطر بیندازد.\n۲

### Markdown webpages

In [36]:
import os
from langchain_core.documents import Document

file_path = '../index/'

docs = []

for markdown_path in texts_paths:

    print(markdown_path)

    with open(markdown_path, "r") as f:
        content = f.read()
    
    if len(content) % 1000 > 0:
        content += " " * (1000 - len(content) % 1000)

    for i in range(0, len(content), 1000):
        chunk = content[i:i + 1000]
        # print(chunk)
        doc = Document(page_content=chunk, metadata={})
        docs.append(doc)


../index/varzesh3.com/news/2167276-برد-نفس-گیر-خاتون-در-دربی-بزرگ-لیگ-زنان.md
../index/varzesh3.com/news/2167196-مهم-ترین-فیفادی-جام-جهانی-در-انتظار-سردار-آزمون.md
../index/varzesh3.com/news/2167143-اعتراف-صریح-رونالدو-از-طاس-شدن-وحشت-دارم.md
../index/varzesh3.com/news/2167027-شوخی-زشت-و-عجیب-با-لیگ-برتر-والیبال-ایران.md
../index/varzesh3.com/news/2167259-مورینیو-باعث-فحاشی-به-من-و-خانواده-ام-شد.md
../index/varzesh3.com/news/2167516-تیم-ملی-خیلی-زود-ریکاوری-کرد.md
../index/varzesh3.com/news/2167364-اشلوت-فقط-با-شانس-لیورپول-را-قهرمان-کرد.md
../index/varzesh3.com/news/2167433-درخواست-وایرال-فوتبالیست-فلسطینی-اجابت-شد.md
../index/varzesh3.com/news/2167329-معجزه-فوتبال-در-مقدماتی-جام-جهانی.md
../index/varzesh3.com/news/2167347-ادامه-اختلاف-در-گچساران-تعداد-کادر-زیاد-است.md
../index/varzesh3.com/news/2167348-پرش-بلند-نعمتی-در-سد-گلر-پاریس-عکس.md
../index/varzesh3.com/news/2167392-زنان-فوتسالیست-آماده-مصاف-با-روسیه.md
../index/varzesh3.com/news/2167257-اولین-بازی-برادر-ستاره-ایتالیا-برای-

In [99]:
import os
from langchain_core.documents import Document

file_path = '../index/'

docs = []

for markdown_file_name in os.listdir(file_path):

    markdown_file_path = file_path+ markdown_file_name
    print(markdown_file_path)

    with open(markdown_file_path, "r") as f:
        content = f.read()
    
    if len(content) % 1000 > 0:
        content += " " * (1000 - len(content) % 1000)

    for i in range(0, len(content), 1000):
        chunk = content[i:i + 1000]
        # print(chunk)
        doc = Document(page_content=chunk, metadata={})
        docs.append(doc)

    # loader = TextLoader(markdown_file_path)
    # documents = loader.load()
    # text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
    # docs.extend(text_splitter.split_documents(documents))


../index/varzesh3.com


IsADirectoryError: [Errno 21] Is a directory: '../index/varzesh3.com'

In [17]:
len(docs)

1

In [ ]:
# pip install unstructured
# pip install "langchain[unstructured]"

# from langchain.document_loaders import UnstructuredMarkdownLoader

# loader = UnstructuredMarkdownLoader("your_file.md")
# documents = loader.load()

# from langchain.text_splitter import RecursiveCharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# chunks = splitter.split_documents(documents)

# # Define the path to your Markdown file
# file_path = "..index/"

# markdown_file_path = "path/to/your/file.md"

# # Initialize the loader
# loader = UnstructuredMarkdownLoader(markdown_file_path)

# # Load the document(s)
# data = loader.load()


# # The 'data' variable will contain a list of LangChain Document objects.
# # If using 'single' mode (default), it will typically be a list with one Document.
# # If using 'elements' mode, it will contain multiple Documents representing elements like headers, paragraphs, etc.
# data

In [18]:
docs = docs[0:300]

In [19]:
len(docs)

1

In [23]:
docs[0].page_content

'## ارلینگ سریعتر از همه به رکورد 50 گل ملی رسید\n# رکورد استثنایی هالند: رونالدو در خطر است!\nارلینگ هالند با هت\u200cتریک در بازی امشب نروژ در ۴۶ بازی از رکورد ۵۰ گل ملی عبور کرد. رکوردی که حتی کریستیانو رونالدو را هم نگران خواهد کرد.\nبه گزارش ورزش سه، \nبرای مقایسه باید اشاره شود که پیش از او هری کین از این نظر رکورددار بود ولی شاهزاده انگلیسی برای رسیدن به ۵۰ گل به ۷۱ بازی نیاز داشت. و کریستیانو رونالدو؟ او گل پنجاهم ملی\u200cاش را در بازی شماره ۱۱۴ و در جام جهانی ۲۰۱۴ مقابل غنا به ثمر رساند.\nالبته فاصله هالند تا رکورد گلزنی ملی رونالدو هنوز زیاد است؛ کاپیتان پرتغال ۱۴۱ گل ملی دارد و فاصله\u200cای بزرگ میان او و هر بازیکن دیگری دیده می\u200cشود که بعید است به این زودی\u200cها این فاصله پر شود. اما نشانه\u200cها روشن\u200cاند: اگر هالند با همین ریتم ادامه دهد، شاید روزی حتی رکورد مردی را که سال\u200cها دست\u200cنیافتنی به نظر می\u200cرسید، به خطر بیندازد.\n۲۵ ساله، بی\u200cرحم در محوطه و تشنه\u200cی گل، ارلینگ هالند نه\u200cفقط در منچسترسیتی، بلکه در \n## اخبار مرتبط\n## دیدگاه\u2

# Embedings

# Vector Store

In [37]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from pydantic import SecretStr

# Set UP the link
vector_store = Chroma(
    collection_name="test",
    embedding_function=OpenAIEmbeddings(
        model="text-embedding-3-small",
        api_key=SecretStr(METISAI_API_KEY),
        base_url="https://api.metisai.ir/openai/v1",
    ),
)

for i in range(int(len(docs) / 250 + 1)):
    vector_store.add_documents(docs[i*250: (i+1)*250], ids=[str(j + i * 250) for j in range(len(docs))])

/var/folders/56/2bx9230j77nckch23fg26xh00000gn/T/ipykernel_3513/1392131427.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [38]:
vector_store._collection.count()

1026

# LLM

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-nano",
    api_key=SecretStr(METISAI_API_KEY),
    base_url="https://api.metisai.ir/openai/v1",
    temperature=0.1,
    verbose=True,
)

In [56]:
a = llm.invoke("ارلینگ هالند چند گل ملی زده است؟")

In [57]:
print(a.content)

تا تاریخ آخرین به‌روزرسانی من در اکتبر 2023، ارلینگ هالند بیش از 30 گل ملی برای تیم ملی نروژ زده است. لطفاً توجه داشته باشید که این آمار ممکن است پس از آن تاریخ تغییر کرده باشد. برای دریافت جدیدترین آمار، می‌توانید به منابع رسمی فدراسیون فوتبال نروژ یا سایت‌های معتبر ورزشی مراجعه کنید.


In [59]:
result = llm.invoke('بازی ایران و روسیه چند چند پایان یافته است؟')
print(result.content)

تا تاریخ آخرین به‌روزرسانی من در اکتبر 2023، اطلاعات دقیقی درباره نتیجه بازی ایران و روسیه در اختیار ندارم. لطفاً به منابع خبری ورزشی معتبر یا سایت‌های رسمی فدراسیون‌های مربوطه مراجعه کنید تا نتیجه دقیق را بیابید.


# Prompt

In [46]:
from langchain import PromptTemplate

# template = """
# You are a fortune teller. These Human will ask you a questions about their life.
# Use following piece of context to answer the question.
# If you don't know the answer, just say you don't know.
# Keep the answer within 2 sentences and concise.

# Context: {context}
# Question: {question}
# Answer:

# """

template = """
لطفا بر اساس مطلب داده شده به سوال پاسخ بده

مطلب: {context}
سوال: {question}
جواب:
"""

# template = """
# لطفا به سوال پرسیده شده پاسخ بده، اگر مطلب ارائه شده مفید بود از آن استفاده کن وگرنه استفاده نکن

# مطلب: {context}
# سوال: {question}
# جواب:
# اگر نمیتونی جواب بدی از دانش خودت یه جواب کوتاه بده
# """

prompt = PromptTemplate(
  template=template,
  input_variables=["context", "question"]
)

In [47]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": vector_store.as_retriever(),  "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

## Without Retriver

In [31]:
template = """
به سوال زیر کوتاه پاسخ بده

سوال: {question}
"""

prompt = PromptTemplate(
  template=template,
  input_variables=["question"]
)

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)


In [55]:
result = rag_chain.invoke("ارلینگ هالند چند گل ملی زده است؟")
print(result)

ارلینگ هالند در حال حاضر ۵۰ گل ملی زده است.


In [58]:
result = rag_chain.invoke("بازی ایران و روسیه چند چند به پایان رسید؟")
print(result)

بر اساس مطالب داده شده، بازی ایران و روسیه در دیدار دوستانه با نتیجه ۲ بر ۱ به نفع روسیه پایان یافته است.


In [51]:
a = vector_store.as_retriever()

In [54]:
a.invoke("ارلینگ هالند")

[Document(metadata={}, page_content='## ارلینگ دو پنالتی را نزد ولی بعد جبران کرد\n# هالند هم انسان است: دو پنالتی خراب شده در یک بازی\nارلینگ هالند امشب با وضعیتی روبرو شد که تا حالا برایش اتفاق نیفتاده بود. از دست دادن دو پنالتی در یک مسابقه.\nبه گزارش ورزش سه، \nهالند این بار تصمیم گرفت به سمت دیگر بزند و توپ را به گوشه بفرستد، اما پرتز دوباره مسیر را درست حدس زد و ضربه را مهار کرد. مهاجم نروژی با نگاهی پر از ناباوری به توپ نگاه می\u200cکرد که از دستان دروازه\u200cبان عبور نکرده بود.\nبا این حال، در همان نیمه اول، هالند جبران کرد و روی پاس دقیق الکساندر سورلوث توپ را دریافت کرده و با ضربه\u200cای زمینی دروازه را باز کرد تا هم دهمین گل خود را در رقابت\u200cهای انتخابی به ثمر برساند و هم نروژ را از شوک بیرون بیاورد.\nهالند در این فصل برای منچسترسیتی متوقف\u200cنشدنی بوده؛ ۱۲ گل در ۹ بازی و در مجموع ۱۳۶ گل در ۱۵۵ مسابقه برای آبی\u200cهای منچستر. شبی که می\u200cتوانست برای هالند کابوس باشد، با یک گل جبران شد چون برای هالند همیشه راهی به سمت دروازه حریفان پیدا می\u200cشود.\n## اخبار مرتب

In [ ]:
result.

'بر اساس مطالب داده شده، اطلاعاتی درباره تعداد گل\u200cهای ملی رونالدو در متن وجود ندارد. بنابراین، نمی\u200cتوانم تعداد گل\u200cهای ملی رونالدو را مشخص کنم.'

In [65]:
result = rag_chain.invoke(" شخصیت Iron Fist در چه سریالی بازی کرده است؟")
print(result)

در سریال "Iron Fist" بازی کرده است.


In [34]:
result = rag_chain.invoke(" چند فصل از سریال Iron Fist تولید شده است؟")
print(result)

در مطلب ذکر شده است که فصل دوم سریال Iron Fist در حال تولید است و پیش از این چند تریلر از فصل دوم منتشر شده است. بنابراین، حداقل یک فصل از این سریال تولید شده است.


In [61]:
result = rag_chain.invoke("سر مربی صربستان کیست؟")
print(result)

بر اساس مطالب داده شده، سرمربی تیم ملی فوتبال صربستان در حال حاضر در حین بازی استعفا داده است و دیگر روی نیمکت حضور ندارد. در متن اشاره شده است که در دقایق پایانی بازی، باتا ژیویکویچ، دستیار مربی، هدایت تیم را بر عهده گرفت. بنابراین، سرمربی فعلی تیم ملی صربستان پس از استعفای دراگان استویکوویچ، باتا ژیویکویچ است.


## Find Accuracy

In [67]:
def answer_question(question):

    return rag_chain.invoke(question)

In [68]:
import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()

development_questions_csv_path = '../DevelopmentQuestions.csv'
development_questions_csv = pd.read_csv(development_questions_csv_path)
# development_questions_csv = development_questions_csv[:20]

development_questions_csv['prediction'] = development_questions_csv['Question'].progress_map(answer_question)
development_questions_csv


  0%|          | 0/40 [00:00<?, ?it/s]

,Question,Answer,prediction
0,موسیقی متن فیلم کرخه تا راین اثر کیست؟,مجید انتظامی,حسین دهلوی
1,فیلم شهر موش ها در چه سالی ساخته شده است؟,۱۳۶۴ شمسی,۱۳۵۸ (1979)
2,اولین فیلم سینمایی که در ایران ساخته شد چه نام...,آبی و رابی,آواز در خواب
3,قدیمی‌ترین سینمای فعال در تهران کدام است؟,سینما فردوسی,سینما فردوسی
4,اولین جشنواره فیلم فجر چه سالی برگزار شد؟,۱۳۶۱ شمسی,۱۳۵۱
5,کدام کارگردان ایرانی بیشترین سیمرغ بلورین جشنو...,مجید مجیدی,اصغر فرهادی
6,اولین فیلم ایرانی که موفق به دریافت جایزه اسکا...,«جدایی نادر از سیمین» به کارگردانی اصغر فرهادی,"فیلم ""جدایی نادر از سیمین"""
7,مشهورترین جشنواره فیلم در جهان کدام است؟,جشنواره فیلم کن,جشنواره فیلم کن
8,کدام کشور بیشترین جوایز اسکار را در زمینه بهتر...,ایتالیا,ایتالیا
9,کدام کارگردان ایرانی به «استاد سینمای ایران» م...,عباس کیارستمی,علی حاتمی


In [69]:
development_questions_csv.iloc[39]['prediction']


'مادر موسی در فیلم "موسی کلیم\u200cالله" نقش\u200cآفرینی نکرده است.'

In [70]:
development_questions_csv.to_csv('output1.csv', index=False)

In [71]:
from evaluator import Evaluator

csv_to_judge_path = 'output1.csv'

evaluator = Evaluator(
    csv_path= csv_to_judge_path,
    question_c='Question',
    correct_answer_c='Answer',
    predicted_answer_c='prediction',
    model_name='First RAG'
)

acc, judged_csv = evaluator.calculate_accuracy()

  0%|          | 0/40 [00:00<?, ?it/s]

First RAG accuracy is: 0.42%


In [76]:
judged_csv.sample(5)

,Question,Answer,prediction,result
39,چه بازیگری نقش مادر موسی در فیلم موسی کلیم الل...,مریلا زارعی,"مادر موسی در فیلم ""موسی کلیم‌الله"" نقش‌آفرینی ...",1
30,جدید ترین فیلم حامد بهداد چیست؟,مفت بر,"جدیدترین فیلم حامد بهداد ""آبادان"" است.",0
25,کارگردان فیلم «خشت و آینه» کیست؟,ابراهیم گلستان,داریوش مهرجویی,0
31,لیلا حاتمی در چه سالی به عنوان داور در جشنوار ...,۲۰۱۴ میلادی,لیلا حاتمی در سال ۱۳۹۸ (2019) به عنوان داور در...,1
37,نام پژمان جمشیدی در فیلم صبحانه با زرافه ها چیست؟,رضا,"در فیلم ""صبحانه با زرافه‌ها""، نام پژمان جمشیدی...",1


In [79]:
print(judged_csv.iloc[37]['prediction'])

در فیلم "صبحانه با زرافه‌ها"، نام پژمان جمشیدی "پژمان" است.
